# Introduction to source control management with Mercurial and Gitlab

## Motivations

### Why source control management?

Source control management (also known as simply "Git" nowadays 🙂) is an essential tool for scientists and developers!

### Why Mercurial?

::::{grid}
:gutter: 2

:::{grid-item}

**Why not Git?** Git is a very good tool for advanced users but it is really too complicated and unsafe for what we need as scientists / students / teachers / developers of most scientific programs.

```{image} ./fig/git.png
:alt: git
:width: 80%
:align: center
```

:::
:::{grid-item}

I (Pierre Augier) think that [Mercurial is more adapted for us in academics](http://www.legi.grenoble-inp.fr/people/Pierre.Augier/mercurial-as-a-great-version-source-control-management-tool-in-academics.html):

- simpler and safer for beginners,

```{image} ./fig/schemes_hggit/simple_commands_hg.png
:alt: git
:width: 80%
:align: center
```

- very powerful for advanced users (it is used for example at Facebook and Mozilla... and for the PyPy repository).

:::

::::

For advanced Git users, when using Mercurial, you lose the Git index (but not what you usually do with it) and you gain very interesting features, like for example long-term named branches, safe distributed history edition (phases, obsolete changesets, evolve extension) and the command `hg absorb` (automatic injection of uncommitted changes into prior commits in a PR).

- https://www.mercurial-scm.org/ 

- https://fluiddyn.readthedocs.io/en/latest/mercurial_heptapod.html

### Why Gitlab?

[Gitlab]: https://about.gitlab.com/

- "[GitLab] is a web-based DevOps lifecycle tool that provides a Git-repository manager providing wiki, issue-tracking and continuous integration and deployment pipeline features, using an open-source license" (from https://en.wikipedia.org/wiki/GitLab).

- [Gitlab] is an open-source and self-hosted alternative to Github (which is not open-source, not self-hosted and now owned by Microsoft).

- Nice and popular projects like Debian and [Gnome](https://gitlab.gnome.org/) use [Gitlab].

- Many public institutions choose Gitlab for their hosting tool, in particular our university UGA has it own [Gitlab] instance: <https://gricad-gitlab.univ-grenoble-alpes.fr>.

```{admonition} Gitlab does not natively support Mercurial!

For now, we'll have to use a Mercurial extension ([hg-git](https://foss.heptapod.net/mercurial/hg-git))
and it won't be as nice as if we could use a platform natively supporting Mercurial.

However, there is now [Heptapod](https://heptapod.net/), a friendly fork of Gitlab with support for Mercurial and Git.
```

## Level 0: command `help`

In [1]:
export HGRCPATH=""

In [2]:
hg help

Mercurial Distributed SCM

list of commands:

Repository creation:

 clone         make a copy of an existing repository
 init          create a new repository in the given directory

Remote repository management:

 incoming      show new changesets found in source
 outgoing      show changesets not found in the destination
 paths         show aliases for remote repositories
 pull          pull changes from the specified source
 push          push changes to the specified destination
 serve         start stand-alone webserver

Change creation:

 commit        commit the specified files or all outstanding changes

Change manipulation:

 backout       reverse effect of earlier changeset
 graft         copy changes from other branches onto the current branch
 merge         merge another revision into working directory

Change organization:

 bookmarks     create a new bookmark or list existing bookmarks
 branch        set or show the current branch name
 branches      list repository name

In [3]:
hg help init

hg init [-e CMD] [--remotecmd CMD] [DEST]

create a new repository in the given directory

    Initialize a new repository in the given directory. If the given directory
    does not exist, it will be created.

    If no directory is given, the current directory is used.

    It is possible to specify an "ssh://" URL as the destination. See 'hg help
    urls' for more information.

    Returns 0 on success.

options:

 -e --ssh CMD       specify ssh command to use
    --remotecmd CMD specify hg command to run on the remote side
    --insecure      do not verify server certificate (ignoring web.cacerts
                    config)

(some details hidden, use --verbose to show complete help)


In [4]:
export HGRCPATH=$PWD/hgrc4ipynb

## Level 0: one local repository

```{image} ./fig/schemes_hggit/simple_commands_hg.png
:alt: git
:width: 60%
:align: center
```

## Commands `init`, `status`, `add`, `remove`, `addremove`, `commit`, `log` and `summary`

In [5]:
# cleanup
rm -rf /tmp/myrepos

Let's create a directory and a file in it.

In [6]:
mkdir -p /tmp/myrepos/myrepo0
cd /tmp/myrepos/myrepo0
touch file0.txt

Let's tell Mercurial that we want to make a new repository from this directory.

In [7]:
hg init

Let's ask Mercurial what is new in this repository.

In [8]:
hg status

? file0.txt


In [9]:
# equivalent command (shorter)
hg st

? file0.txt


We want to "Schedule files to be version controlled and added to the repository."

In [10]:
hg add

adding file0.txt


A new `status` gives

In [11]:
hg st

A file0.txt


Let's now create the first commit (or "changeset"). We can think about it as **a snapshot of the code** as it is now.

In [12]:
hg commit -m "First commit"

A new `status` returns nothing because there is nothing new from the last commit.

In [13]:
hg st

Let's add a new file:

In [14]:
touch file1.txt

In [15]:
hg st

? file1.txt


In [16]:
rm -f file0.txt

In [17]:
hg st

! file0.txt
? file1.txt


One new file and one file removed from the working directory. We want to take a new snapshot of the code how it is now. `addremove` is very useful is that common situation.

In [18]:
hg addremove
# or
# hg addre

removing file0.txt
adding file1.txt


In [19]:
hg st

A file1.txt
R file0.txt


In [20]:
hg commit -m "Remove file0 and add file1"

In [21]:
hg st

In [22]:
ls

file1.txt


Let's modify the file `file1.txt`: 

In [23]:
echo "Hello world!" >> file1.txt

In [24]:
hg st

M file1.txt


There is no "Git index" in Mercurial. A modification in a versioned file is automatically included in the next commit. 

We can really think about the commits as "snapshots of the code" as it is in the working directory.

In [25]:
hg commit -m "Hello world in file1.txt"

In [26]:
hg st

We used a lot `status` to get information about the state of the repository. Let's discover 2 other very useful commands: `sum` (or `summary`) and `log`.

In [27]:
hg sum

parent: 2:8bfba1b25e7d tip
 Hello world in file1.txt
branch: default
commit: (clean)
update: (current)
phases: 3 draft


Mercurial tells us that we are in the branch `default`, which is... the default "named branch". Note that Mercurial "named branches" are used only for long term features or versions of software (for example "pypy2.7" and "pypy3.6" in Pypy repository) and not for feature branches like branches in Git. For this tutorial, we won't use other named branches and we'll always work in the `default` branch.

```{margin} Branches

When reading stuff on the web or working in open-source, you will hear a lot about "branches". Most of the time, it will be about "Git branches", which are lightweight "feature branches".

Mercurial has different concepts to handle different types of "branches":

- named branches (for long term features or versions of software, see `hg help branch`)

- anonymous branches, which are created when one creates a commit from a commit which has already a child.

- bookmarks (for "feature branches", see `hg help bookmark`). This is the closest equivalent to the "Git branches" and it is used to represent "Git branches" when working with hg-git.

- topics (with the topic extension, see [this tutorial](https://www.mercurial-scm.org/doc/evolution/tutorials/topic-tutorial.html)). Better than bookmarks. 
This is the modern way to work with "feature branches" with Mercurial.

```

In [28]:
hg log

changeset:   2:8bfba1b25e7d
tag:         tip
user:        Pierre Augier <pa@example.com>
date:        Tue Sep 06 21:44:41 2022 +0200
summary:     Hello world in file1.txt

changeset:   1:02ac2678cff1
user:        Pierre Augier <pa@example.com>
date:        Tue Sep 06 21:44:40 2022 +0200
summary:     Remove file0 and add file1

changeset:   0:f7dcc9188623
user:        Pierre Augier <pa@example.com>
date:        Tue Sep 06 21:44:38 2022 +0200
summary:     First commit



In [29]:
hg log --graph

@  changeset:   2:8bfba1b25e7d
|  tag:         tip
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:41 2022 +0200
|  summary:     Hello world in file1.txt
|
o  changeset:   1:02ac2678cff1
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:40 2022 +0200
|  summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



In [30]:
hg log -G

@  changeset:   2:8bfba1b25e7d
|  tag:         tip
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:41 2022 +0200
|  summary:     Hello world in file1.txt
|
o  changeset:   1:02ac2678cff1
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:40 2022 +0200
|  summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



## Level 0.1: about the special files .hgignore and .gitignore

There are files that you don't want to include in a repository. Usually, you want to include the source but not what is produced with the source (programs, libraries, pdf files, etc.).

To avoid to have too many files listed when one call `hg st`, one can add a file `.hgignore` (or `.gitignore`, when using hg-git).

See https://www.mercurial-scm.org/wiki/.hgignore and https://git-scm.com/docs/gitignore

Typical content of a `.gitignore` file:

```
*.pyd*
*.pyc
*~
*tmp
\#*\#
*.slides.html
.ipynb_checkpoints
```

## Level 1.0: command `revert`

In [31]:
cat file1.txt

Hello world!


In [32]:
echo "Something wrong" > file1.txt

Oh, here, we really did something wrong. It's buggy. Let's see where is our problem.

In [33]:
hg st

M file1.txt


The command `diff` shows the differences between the working directory and the parent commit.

In [34]:
hg diff

diff --git a/file1.txt b/file1.txt
--- a/file1.txt
+++ b/file1.txt
@@ -1,1 +1,1 @@
-Hello world!
+Something wrong


We could also use `hg meld` which opens the program `meld` to present the same thing graphically.

In [35]:
hg revert file1.txt
# or to revert all files:
# hg revert --all

In [36]:
ls

file1.txt  file1.txt.orig


In [37]:
hg st

? file1.txt.orig


In [38]:
rm -f file1.txt.orig

## Level 1.1: command `update`

In [39]:
hg log -G

@  changeset:   2:8bfba1b25e7d
|  tag:         tip
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:41 2022 +0200
|  summary:     Hello world in file1.txt
|
o  changeset:   1:02ac2678cff1
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:40 2022 +0200
|  summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



In [40]:
hg update 0
# `update` or just `up`

1 files updated, 0 files merged, 1 files removed, 0 files unresolved


In [41]:
hg sum

parent: 0:f7dcc9188623 
 First commit
branch: default
commit: (clean)
update: 2 new changesets (update)
phases: 3 draft


In [42]:
ls

file0.txt


In [43]:
echo "A line in file0.txt" >> file0.txt

In [44]:
hg st

M file0.txt


In [45]:
hg commit -m "A line in file0.txt"

created new head


We were at commit 0 and we commited. It created a (unnamed) branch.

In [46]:
hg log -G

@  changeset:   3:d7f0bba98ea8
|  tag:         tip
|  parent:      0:f7dcc9188623
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:46 2022 +0200
|  summary:     A line in file0.txt
|
| o  changeset:   2:8bfba1b25e7d
| |  user:        Pierre Augier <pa@example.com>
| |  date:        Tue Sep 06 21:44:41 2022 +0200
| |  summary:     Hello world in file1.txt
| |
| o  changeset:   1:02ac2678cff1
|/   user:        Pierre Augier <pa@example.com>
|    date:        Tue Sep 06 21:44:40 2022 +0200
|    summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



In [47]:
hg up 1

1 files updated, 0 files merged, 1 files removed, 0 files unresolved


In [48]:
hg sum

parent: 1:02ac2678cff1 
 Remove file0 and add file1
branch: default
commit: (clean)
update: 2 new changesets (update)
phases: 4 draft


In [49]:
hg up

1 files updated, 0 files merged, 0 files removed, 0 files unresolved
updated to "8bfba1b25e7d: Hello world in file1.txt"
1 other heads for branch "default"


In [50]:
hg sum

parent: 2:8bfba1b25e7d 
 Hello world in file1.txt
branch: default
commit: (clean)
update: 1 new changesets, 2 branch heads (merge)
phases: 4 draft


In [51]:
ls

file1.txt


In [52]:
touch file2.txt

In [53]:
hg st

? file2.txt


In [54]:
hg add

adding file2.txt


In [55]:
hg commit -m "Add file 2"

In [56]:
hg log -G

@  changeset:   4:bd539f0ab390
|  tag:         tip
|  parent:      2:8bfba1b25e7d
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:49 2022 +0200
|  summary:     Add file 2
|
| o  changeset:   3:d7f0bba98ea8
| |  parent:      0:f7dcc9188623
| |  user:        Pierre Augier <pa@example.com>
| |  date:        Tue Sep 06 21:44:46 2022 +0200
| |  summary:     A line in file0.txt
| |
o |  changeset:   2:8bfba1b25e7d
| |  user:        Pierre Augier <pa@example.com>
| |  date:        Tue Sep 06 21:44:41 2022 +0200
| |  summary:     Hello world in file1.txt
| |
o |  changeset:   1:02ac2678cff1
|/   user:        Pierre Augier <pa@example.com>
|    date:        Tue Sep 06 21:44:40 2022 +0200
|    summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



We cleanup a bit to get back a linear history. Since all commits are local, they are in a draft phase and they can be obsoleted without problem.

In [57]:
hg prune 3

1 changesets pruned


In [58]:
hg log -G

@  changeset:   4:bd539f0ab390
|  tag:         tip
|  parent:      2:8bfba1b25e7d
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:49 2022 +0200
|  summary:     Add file 2
|
o  changeset:   2:8bfba1b25e7d
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:41 2022 +0200
|  summary:     Hello world in file1.txt
|
o  changeset:   1:02ac2678cff1
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:40 2022 +0200
|  summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



## Level 2: working with a remote repository (usually on the web)

```{image} ./fig/schemes_hggit/simple_commands_hg.png
:alt: git
:width: 60%
:align: center
```

### Commands `clone`, `push`, `pull`

The most standard way to create a local repository is to clone a remote repository available on the web. This is done with the command `clone` like this:

```bash
hg clone ssh://hg@foss.heptapod.net/fluiddyn/fluidsim
```

Here, I used ssh, which is usually more convenient than https, but one can also use http addresses like `https://github.com/hpyproject/hpy.git`. On Github, Gitlab, Heptapod, there is usually a blue button `Code` or `Clone` from which one can get the right address to clone a repository. 

Warning: sometimes, the addresses are not exactly like the web address, for example `git@github.com:hpyproject/hpy.git`.

Note: with the extension `hggit`, one can work on remote Git repositories with a local Mercurial repository (more on that later).

For this tutorial, we will create a local "web" repository.

In [59]:
cd ..
pwd

/tmp/myrepos


In [60]:
hg clone myrepo0 myrepo_web

requesting all changes
adding changesets
adding manifests
adding file changes
added 4 changesets with 4 changes to 3 files
1 new obsolescence markers
new changesets f7dcc9188623:bd539f0ab390
updating to branch default
2 files updated, 0 files merged, 0 files removed, 0 files unresolved


Now, we have a "web" repository that we can use as if it was on the web. Let's first clone it.

In [61]:
hg clone myrepo_web myrepo1

updating to branch default
2 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [62]:
cd myrepo1

In [63]:
ls

file1.txt  file2.txt


In [64]:
echo "Hello" > file2.txt
hg st

M file2.txt


In [65]:
hg commit -m "Improve file2"

In [66]:
hg log -G

@  changeset:   4:c3c17f042f45
|  tag:         tip
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:51 2022 +0200
|  summary:     Improve file2
|
o  changeset:   3:bd539f0ab390
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:49 2022 +0200
|  summary:     Add file 2
|
o  changeset:   2:8bfba1b25e7d
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:41 2022 +0200
|  summary:     Hello world in file1.txt
|
o  changeset:   1:02ac2678cff1
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:40 2022 +0200
|  summary:     Remove file0 and add file1
|
o  changeset:   0:f7dcc9188623
   user:        Pierre Augier <pa@example.com>
   date:        Tue Sep 06 21:44:38 2022 +0200
   summary:     First commit



In [67]:
hg push

pushing to /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 1 changesets with 1 changes to 1 files


At this point, we have 2 repositories.

In [68]:
cd /tmp/myrepos
pwd
ls

/tmp/myrepos
myrepo0  myrepo1  myrepo_web


Let's create a new repository from the repo on the web. (Usually, it is done in another computer.)

In [69]:
hg clone myrepo_web myrepo_other_computer

updating to branch default
2 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [70]:
cd myrepo_other_computer
ls

file1.txt  file2.txt


In [71]:
cat file2.txt

Hello


In [72]:
echo "Hello Word!" > file2.txt

In [73]:
hg st

M file2.txt


In [74]:
hg commit -m '"Hello Word!" is even nicer'

In [75]:
hg st

We push the new changeset towards the default repository (from which this repository has been created), which is the "web" repository.

In [76]:
hg push

pushing to /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 1 changesets with 1 changes to 1 files


Let's come back to the original repository in the first computer.

In [77]:
cd ../myrepo1

In [78]:
cat file2.txt

Hello


In [79]:
hg sum

parent: 4:c3c17f042f45 tip
 Improve file2
branch: default
commit: (clean)
update: (current)


In [80]:
hg pull

pulling from /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 1 changesets with 1 changes to 1 files
new changesets d416bb8bde96
(run 'hg update' to get a working copy)


In [81]:
hg up

1 files updated, 0 files merged, 0 files removed, 0 files unresolved


We could have just run `hg pull -u` to pull and update to the tip of the branch where we are, i.e. the last commit since the repository is now in a linear state.

In [82]:
hg sum

parent: 5:d416bb8bde96 tip
 "Hello Word!" is even nicer
branch: default
commit: (clean)
update: (current)


In [83]:
cat file2.txt

Hello Word!


## Level 3: nonlinear history, unnamed branches and `merge`/`rebase` commands  

We can now consider the case in which 2 persons (Alice and Bob) work at the same time on the same project. First they both clone the web repository.

In [84]:
# Alice
cd /tmp/myrepos/
hg clone myrepo_web myrepo_Alice

updating to branch default
2 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [85]:
# Bob
hg clone myrepo_web myrepo_Bob

updating to branch default
2 files updated, 0 files merged, 0 files removed, 0 files unresolved


Just for this notebook, we can tell Mercurial who is the owner of Alice's and Bob's repositories. 

In [86]:
# not useful in real life!
echo "username = Alice <alice@mail.wonderland>" >> /tmp/myrepos/myrepo_Alice/.hg/hgrc
echo "username = Bob <bob@mail.wonderland>" >> /tmp/myrepos/myrepo_Bob/.hg/hgrc

In [87]:
# Alice works
cd /tmp/myrepos/myrepo_Alice
touch Alice_file.py
hg add
hg commit -m "Alice adds a file"
hg push

adding Alice_file.py
pushing to /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 1 changesets with 1 changes to 1 files


In [88]:
# Bob works
cd /tmp/myrepos/myrepo_Bob
touch Bob_file.py
hg add
hg commit -m "Bob adds a file"

adding Bob_file.py


At this point, `hg push` would lead to the following error:

```
pushing to /tmp/myrepos/myrepo_web
searching for changes
remote has heads on branch 'default' that are not known locally: 69b3f39e70ab
abort: push creates new remote head 938fc2084593!
(pull and merge or see 'hg help push' for details about pushing new heads)
```

Bob encounters a problem... The remote repository has a commit which is not know locally. Let's first pull this commit.

In [89]:
hg pull

pulling from /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 1 changesets with 1 changes to 1 files (+1 heads)
new changesets faaf587b5bd9
(run 'hg heads' to see heads, 'hg merge' to merge)


Now we can examine the situation.

In [90]:
hg heads

changeset:   7:faaf587b5bd9
tag:         tip
parent:      5:d416bb8bde96
user:        Alice <alice@mail.wonderland>
date:        Tue Sep 06 21:44:57 2022 +0200
summary:     Alice adds a file

changeset:   6:39d0c9e5a69f
user:        Bob <bob@mail.wonderland>
date:        Tue Sep 06 21:44:58 2022 +0200
summary:     Bob adds a file



In [91]:
hg log -G -l 4

o  changeset:   7:faaf587b5bd9
|  tag:         tip
|  parent:      5:d416bb8bde96
|  user:        Alice <alice@mail.wonderland>
|  date:        Tue Sep 06 21:44:57 2022 +0200
|  summary:     Alice adds a file
|
| @  changeset:   6:39d0c9e5a69f
|/   user:        Bob <bob@mail.wonderland>
|    date:        Tue Sep 06 21:44:58 2022 +0200
|    summary:     Bob adds a file
|
o  changeset:   5:d416bb8bde96
|  user:        Pierre Augier <pa@example.com>
|  date:        Tue Sep 06 21:44:53 2022 +0200
|  summary:     "Hello Word!" is even nicer
|
o  changeset:   4:c3c17f042f45
|  user:        Pierre Augier <pa@example.com>
~  date:        Tue Sep 06 21:44:51 2022 +0200
   summary:     Improve file2



We have 2 "unnamed branches" (in the "named branch" `default`), i.e. 2 different commits with the same parent (changeset 5).

A simple way to avoid this situation is to pull just before a commit and to make sure to commit from the last commit in the shared repository.

There are different ways to solve this problem. The nicest would be to move changeset 6 above changeset 7 (actually change the parent of changeset 6 to changeset 7). It's very simple to do with the command `hg rebase -s 6 -d 7` (`-s` and `-d` mean source and destination, respectively). It is nice since it leads to a linear state but it is a little bit too complicated (history rewriting).

In the error log, Mercurial tells us about the simple way to solve the situation: pull and merge. Let's try that.

In [92]:
hg merge

1 files updated, 0 files merged, 0 files removed, 0 files unresolved
(branch merge, don't forget to commit)


In [93]:
hg sum

parent: 6:39d0c9e5a69f 
 Bob adds a file
parent: 7:faaf587b5bd9 tip
 Alice adds a file
branch: default
commit: 1 modified (merge)
update: (current)
phases: 1 draft


In [94]:
hg st

M Alice_file.py
# The repository is in an unfinished *merge* state.

# To continue:    hg commit
# To abort:       hg merge --abort



In [95]:
ls

Alice_file.py  Bob_file.py  file1.txt  file2.txt


In [96]:
hg commit -m "Merge Alice and Bob commits"

In [97]:
hg log -G -l 4

@    changeset:   8:ba60c7f77030
|\   tag:         tip
| |  parent:      6:39d0c9e5a69f
| |  parent:      7:faaf587b5bd9
| |  user:        Bob <bob@mail.wonderland>
| |  date:        Tue Sep 06 21:45:00 2022 +0200
| |  summary:     Merge Alice and Bob commits
| |
| o  changeset:   7:faaf587b5bd9
| |  parent:      5:d416bb8bde96
| |  user:        Alice <alice@mail.wonderland>
| |  date:        Tue Sep 06 21:44:57 2022 +0200
| |  summary:     Alice adds a file
| |
o |  changeset:   6:39d0c9e5a69f
|/   user:        Bob <bob@mail.wonderland>
|    date:        Tue Sep 06 21:44:58 2022 +0200
|    summary:     Bob adds a file
|
o  changeset:   5:d416bb8bde96
|  user:        Pierre Augier <pa@example.com>
~  date:        Tue Sep 06 21:44:53 2022 +0200
   summary:     "Hello Word!" is even nicer



Note that the history is no longer linear. However, the repository is now in a good state so we should be able to push in the remote repository.

In [98]:
hg push

pushing to /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 2 changesets with 1 changes to 1 files


And Alice can get Bob's file.

In [99]:
# Alice works
cd /tmp/myrepos/myrepo_Alice
hg pull -u

pulling from /tmp/myrepos/myrepo_web
searching for changes
adding changesets
adding manifests
adding file changes
added 2 changesets with 1 changes to 1 files
new changesets 39d0c9e5a69f:ba60c7f77030
1 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [100]:
hg sum

parent: 8:ba60c7f77030 tip
 Merge Alice and Bob commits
branch: default
commit: (clean)
update: (current)


In [101]:
ls

Alice_file.py  Bob_file.py  file1.txt  file2.txt


## Working with Gitlab

We are going to use the UGA instance <https://gricad-gitlab.univ-grenoble-alpes.fr>. 

- Login on the website.
- From the website, create a repository
- Clone it with Mercurial locally (once again, you need to activate the extension `hggit` enabled in the user Mercurial config file).

  Note the difference between the https and ssh addresses. To use ssh, one first has to [setup a ssh key](https://gricad-gitlab.univ-grenoble-alpes.fr/profile/keys).
  Let's do it! It is easy and very useful!
  
- Create a `README.md` file and fill it with few lines of markdown code.

- Commit and push:

```
hg st
hg add
# create a commit locally on your computer
hg commit -m "Modify README.md"
# push it on the web server
hg push
```


## Mercurial configuration files

Simple text files... They can be edited with any text editors.

- `~/.hgrc` (per user)

If you don't have this file, you can create a reasonable conf. file with `hg config --edit`. See https://fluiddyn.readthedocs.io/en/latest/mercurial_heptapod.html#set-up-mercurial. This file contains in particular the list of activated Mercurial extensions.

To use Mercurial with Git repositories hosted on Github/Gitlab, one need at least (yes, nothing after the `=`):

```
[extensions]
hggit =
```

- `my_repo/.hg/hgrc` (per repo)

This file contains in particular the path from which to pull/push by default. Something like:

```
[paths]
default = ssh://hg@foss.heptapod.net/fluiddyn/transonic
```

To work with hg-git, it is convenient to have something like:

```
[paths]
default = git@github.com:paugier/pythran.git
upstream = https://github.com/serge-sans-paille/pythran.git
```


## Standard Github / Gitlab workflow

```{image} ./fig/schemes_hggit/github_workflow.png
:alt: git
:width: 70%
:align: center
```

Git branches are really like "bookmarks" that point towards a commit. In Mercurial, such things are called "bookmarks".

::::{margin}
:::{admonition} Name of the default Git branch: `master` or `main`?

Historically, the default branch in a Git repository was called `master`. But this word is associated with slavery and now the default branch tends to be called `main`. Nowadays, one needs to check manually which name is used for the default branch (`master` or `main`).
:::
::::

To create a Git branch in your remote forked repository, one can do:

```bash
hg pull upstream
# update the code to the last commit of the `main` Git branch
hg up main
# create a new bookmark for our new Git branch
hg bookmark my-fix-or-new-feature
hg commit
# ...
hg push -B my-fix-or-new-feature
```

Then, one can create the Pull Request (Github) or Merge Request (Gitlab). Usually, it it done through the web interface.

## Heptapod workflow (without forks)

```{image} ./fig/schemes_hggit/heptapod_workflow.png
:alt: git
:width: 70%
:align: center
```

Mercurial feature branches are called "topics".

To create a Merge Request in the remote repository, one can do:

```bash
hg pull
hg up default
hg topic my-fix-or-new-feature
hg commit
# ...
hg push
```
